In [ ]:
import pickle
import os
import re
from pathlib import Path

from trinity.utils.visqa import parse_value

### Change the `result_dir` to the corresponding folder that you want to analyze

In [ ]:
result_dir, path_plate = "./logs/full-1119/", "{}/{}_full.log"
# result_dir, path_plate = "./logs/abstract-only-1119/", "{}/{}_abstract-only.log"
# result_dir, path_plate = "./logs/optimal-only-1119/", "{}/{}_optimal-only.log"

data_path = "./benchmarks/VisQA/shared/"

In [ ]:
with open("{}/tapas_on_visqa_dataset.pkl".format(data_path), "rb") as f:
    dt = pickle.load(f)

In [ ]:
# for every benchmark, search for logs
collected_answers = []
expected_answers = []
n_timeout = 0
for i in range(len(dt)):
    tmp_id = dt[i]["short_id"]
    expected_answers.append(dt[i]["repr_answer"])
    tmp_log_path = path_plate.format(result_dir, tmp_id)
    # test file existence
    tmp_pflag = os.path.exists(tmp_log_path)
    if tmp_pflag:
        # found, look for review report ranking list
        # read logs
        with open(tmp_log_path, "r") as f:
            tmp_log = f.read()
            
        if "---------- timeout ----------" in tmp_log:
            n_timeout += 1
            print("# i={}, {}, timeout detected".format(i, dt[i]["short_id"]))
            
        # first detect for fallback strategy (original)
        if "# ====> fallback strategy (original) is triggered" in tmp_log:
        # if True:
            # collected_answers.append([(-1,dt[i]["repr_answer"])])
            collected_answers.append([(-1,[dt[i]["candidate_outputs"]["TaPas_original"][0]])])
            # collected_answers.append(None)
            pass
        else:
            tmp0_res = sorted(
                re.findall(r"# top-(.*?), score: (.*?), answer: (.*)", tmp_log),
                key=lambda x:x[1],
                reverse=True,
            )

            # ========
            # FIXME: for new version of logs, you need to evaluate it to get the repr_answer
            # ========

            # sort out and remove duplicates
            tmp1_res = []
            tmp1_res_set = set()
            for p in tmp0_res:
                if p[2] not in tmp1_res_set:
                    # don't include the same answer twice
                    # tmp1_res_set.add(p[2])
                    # tmp1_res.append((p[1],[p[2]])) # (top score, repr_answer)
                    tmp1_res_set.add(p[2])
                    tmp1_res.append((p[1],eval(p[2]))) # (top score, repr_answer)
            collected_answers.append(tmp1_res)
    else:
        # probably not ready, put None
        collected_answers.append(None)

In [ ]:
# compute top-1 accuracy
# ========
# FIXME: for new version of logs, you need new methods to automatically compute this
# ========
def parse_list(arg_list):
    Q = [parse_value(str(p).lower()) for p in arg_list]
    return [str(p) if isinstance(p,str) else "{:.4f}".format(p) for p in Q]

N_TOP = 1
acc = 0
tot = 0
for i in range(len(dt)):
    if collected_answers[i] is None:
        print("i={}, -1".format(i))
        # not ready
        continue
    
    tot += 1
    proposed_answers = [p[1] for p in collected_answers[i][:N_TOP]]
    proposed_scores = [p[0] for p in collected_answers[i][:N_TOP]]
    is_found = False
    for j in range(len(proposed_answers)):
        p = proposed_answers[j]
        s = proposed_scores[j]
#         pp = parse_list(p)
#         pe = parse_list(expected_answers[i])
#         print("i={}, pp={}, pe={}".format(i,pp,pe))
        if parse_list(p) == parse_list(expected_answers[i]):
            acc += 1
            is_found = True
#             if s==-1:
#                 print("# i={}, {}, hit, original fallback".format(i, dt[i]["short_id"]))
#             else:
#                 print("# i={}, {}, hit".format(i, dt[i]["short_id"]))
            break
    if is_found:
        print("1")
    else:
        print("0")

In [ ]:
acc, tot

In [ ]:
acc/tot

In [ ]:
n_timeout